In [74]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from dataloader import JPXData
import helper_functions as hf
import matplotlib.pyplot as plt
from autoencoder import autoencoder as ae

In [75]:
stock_prices = pd.read_csv("modified_data/autoencoder_data.csv")

In [76]:
# ~ 3 month testing time frame 2022-01-04 to 2022-04-28
train_data = stock_prices[stock_prices["Date"] < '2022-01-04'].copy().reset_index(drop=True)
test_data  = stock_prices[stock_prices["Date"] >= '2022-01-04'].copy().reset_index(drop=True)

In [77]:
train_class = JPXData(data=train_data, model="ae")
trainloader = torch.utils.data.DataLoader(train_class, batch_size=1, shuffle=True, num_workers=5)
test_class  = JPXData(data=test_data, model="ae")
testloader  = torch.utils.data.DataLoader(test_class, batch_size=1, shuffle=True, num_workers=5)

In [78]:
net = ae(e_layers=[69, 60, 50, 40, 30, 25, 20], d_layers=[20, 25, 30, 40, 50, 60, 69]).cuda()

In [79]:
criterion = nn.MSELoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=1e-4, momentum=0.9)

In [80]:
loss_a1 = []

for epoch in range(400):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels, date = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = hf.loss_function(outputs, labels, criterion=nn.L1Loss().cuda())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
        if i % 1000 == 999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 1000:.10f}')
            loss_a1.append(running_loss/1000)
            running_loss = 0.0
        # torch.save(net.state_dict(), f'saved_models/{epoch}.pt')

print('Finished Training')

[1,  1000] loss: 0.8017404274
[2,  1000] loss: 0.7986440708
[3,  1000] loss: 0.7953418345
[4,  1000] loss: 0.7869931201
[5,  1000] loss: 0.7793430185
[6,  1000] loss: 0.7672326139
[7,  1000] loss: 0.7471453162
[8,  1000] loss: 0.7222991506
[9,  1000] loss: 0.6924631490
[10,  1000] loss: 0.6590671191
[11,  1000] loss: 0.6248017327
[12,  1000] loss: 0.5961570278
[13,  1000] loss: 0.5692612517
[14,  1000] loss: 0.5493568161
[15,  1000] loss: 0.5345583777
[16,  1000] loss: 0.5258881842
[17,  1000] loss: 0.5190865810
[18,  1000] loss: 0.5124980113
[19,  1000] loss: 0.5072674504
[20,  1000] loss: 0.5021690064
[21,  1000] loss: 0.4987980314
[22,  1000] loss: 0.4971070869
[23,  1000] loss: 0.4886905581
[24,  1000] loss: 0.4858840983
[25,  1000] loss: 0.4835191741
[26,  1000] loss: 0.4800120410
[27,  1000] loss: 0.4780312824
[28,  1000] loss: 0.4783280729
[29,  1000] loss: 0.4768804399
[30,  1000] loss: 0.4735235122
[31,  1000] loss: 0.4733694286
[32,  1000] loss: 0.4717417294
[33,  1000] loss:

In [ ]:
final_loss = np.array(loss_a1)/2
plt.plot(final_loss)
plt.show()